In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


data = pd.read_csv(r'./predict-student-performance-from-game-play/train.csv')
#data_label = pd.read_csv(r'./predict-student-performance-from-game-play/train_labels.csv')
data.head(100)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20090312431273200,95,98859,person_click,basic,2,NaN,277.753433,-166.585120,613.0,444.0,NaN,Could be. But we need evidence!,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.found,0,0,1,0-4
96,20090312431273200,96,99459,person_click,basic,2,NaN,368.432843,-176.661681,676.0,451.0,NaN,Why don't you head to the Basketball Center an...,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.found,0,0,1,0-4
97,20090312431273200,97,99958,person_click,basic,2,NaN,368.432120,-176.661922,676.0,451.0,NaN,Sure!,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.found,0,0,1,0-4
98,20090312431273200,98,100713,person_click,basic,2,NaN,400.098469,-176.662028,698.0,451.0,NaN,I'll be at the Capitol. Let me know if you fin...,gramps,tunic.historicalsociety.collection,tunic.historicalsociety.collection.gramps.found,0,0,1,0-4


In [3]:
pd.set_option('display.max_rows', 1000)
print(data['room_fqid'].unique())
data['elapsed_time_diff'] = 0 - data.groupby('session_id')['elapsed_time'].diff(periods = -1)
data = data.dropna(subset= 'elapsed_time_diff', axis=0, how = 'any')

def replace_negative_with_median(group, col):
    median_value = group[col].median()
    group[col] = group[col].apply(lambda x: x if x >= 0 else median_value)
    return group

df1 = data[data['event_name'] != 'checkpoint'].groupby(['session_id', 'level_group']).apply(replace_negative_with_median, col = 'elapsed_time_diff')
df1['elapsed_time_diff'] = pd.to_numeric(df1['elapsed_time_diff'], errors = 'coerce')

['tunic.historicalsociety.closet' 'tunic.historicalsociety.basement'
 'tunic.historicalsociety.entry' 'tunic.historicalsociety.collection'
 'tunic.historicalsociety.stacks' 'tunic.kohlcenter.halloffame'
 'tunic.capitol_0.hall' 'tunic.historicalsociety.closet_dirty'
 'tunic.historicalsociety.frontdesk' 'tunic.humanecology.frontdesk'
 'tunic.drycleaner.frontdesk' 'tunic.library.frontdesk'
 'tunic.library.microfiche' 'tunic.capitol_1.hall'
 'tunic.historicalsociety.cage' 'tunic.historicalsociety.collection_flag'
 'tunic.wildlife.center' 'tunic.flaghouse.entry' 'tunic.capitol_2.hall']


In [10]:
df1.groupby(['session_id','level_group','room_fqid']).agg({'elapsed_time_diff':'sum'})
df1_modified = df1.groupby(['session_id','level_group','event_name']).agg({'elapsed_time_diff':'sum'})
df1_modified = df1_modified.reset_index()
session_event_elapsed_time = df1_modified.pivot(index = ['session_id','level_group'], columns = 'event_name', values = 'elapsed_time_diff')[['navigate_click','person_click','cutscene_click','object_click']]
session_event_elapsed_time.to_csv('session_event_elapsed_time.csv')
session_event_elapsed_time

event_name                     navigate_click  person_click  cutscene_click  \
session_id        level_group                                                 
20090312431273200 0-4                113882.0       17111.0         27593.0   
                  13-22              211657.0      100300.0         63760.0   
                  5-12               116046.0       82412.0          8031.0   
20090312433251036 0-4                109679.0       33148.0         49324.5   
                  13-22             1044345.0      201384.5        121897.0   
...                                       ...           ...             ...   
22100219442786200 13-22              225166.0      118279.0         81881.0   
                  5-12               112963.0       99593.0          9363.0   
22100221145014656 0-4                195863.5       60874.0         77745.0   
                  13-22              485898.0      267495.0        149036.0   
                  5-12               328390.0      375467.0         26250.0   

event_name                     object_click  
session_id        level_group                
20090312431273200 0-4               13629.0  
                  13-22             21486.0  
                  5-12              32912.0  
20090312433251036 0-4               19618.5  
                  13-22            121392.5  
...                                     ...  
22100219442786200 13-22             23233.0  
                  5-12              31536.0  
22100221145014656 0-4               46361.5  
                  13-22             49136.0  
                  5-12             223840.0  

[70686 rows x 4 columns]

In [11]:
d2 = data.groupby(['session_id','level_group','event_name']).agg({'event_name':'count'})
sum_by_session_level = d2.groupby(level=['session_id','level_group']).sum()
d3 = (d2/sum_by_session_level).rename(columns = {'event_name':'frequency'})
d4 = d3.reset_index().pivot(index = ['session_id','level_group'], columns = 'event_name', values = 'frequency')[['navigate_click','person_click','cutscene_click','object_click']].rename(columns = {'navigate_click':'navigate_click_freq',\
'person_click':'person_click_freq','cutscene_click':'cutscene_click_freq','object_click':'object_click_freq'})
d4_0_4 = d4[d4.index.get_level_values('level_group') == '0-4']
d4_5_12 = d4[d4.index.get_level_values('level_group') == '5-12']
d4_13_22 = d4[d4.index.get_level_values('level_group') == '13-22']
d4_0_4.to_csv('session_event_frequency_0_4.csv')
d4_5_12.to_csv('session_event_frequency_5_12.csv')
d4_13_22.to_csv('session_event_frequency_13_22.csv')
d4_13_22

,event_name,navigate_click_freq,person_click_freq,cutscene_click_freq,object_click_freq
session_id,level_group,,,,
20090312431273200,13-22,0.405728,0.293556,0.143198,0.047733
20090312433251036,13-22,0.491512,0.111883,0.050154,0.064043
20090312455206810,13-22,0.338843,0.278237,0.159780,0.030303
20090313091715820,13-22,0.477551,0.224490,0.134694,0.036735
20090313571836404,13-22,0.437340,0.273657,0.140665,0.038363
...,...,...,...,...,...
22100215342220508,13-22,0.399527,0.257683,0.139480,0.052009
22100215460321130,13-22,0.326711,0.298013,0.154525,0.028698
22100217104993650,13-22,0.456486,0.208539,0.123153,0.032841


In [13]:
df1.groupby(['session_id','level_group','room_fqid']).agg({'elapsed_time_diff':'sum'}).to_csv('session_room_elapsed_time.csv')
df1.groupby(['session_id','level_group','event_name']).agg({'elapsed_time_diff':'sum'}).to_csv('session_event_elapsed_time.csv')
d2 = data.groupby(['session_id','level_group','event_name']).agg({'event_name':'count'})
sum_by_session_level = d2.groupby(level=['session_id','level_group']).sum()
(d2/sum_by_session_level.rename({'event_name':'frequency'}))

event_name
session_id        level_group event_name                    
20090312431273200 0-4         checkpoint            0.006061
                              cutscene_click        0.169697
                              map_click             0.012121
                              map_hover             0.024242
                              navigate_click        0.490909
...                                                      ...
22100221145014656 5-12        notification_click    0.015625
                              object_click          0.208333
                              object_hover          0.085069
                              observation_click     0.003472
                              person_click          0.232639

[711592 rows x 1 columns]